In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import skimage
from skimage import filters, io, transform, exposure, morphology, segmentation
from pandas import DataFrame as df
from scipy import ndimage

In [ ]:
imc = skimage.io.imread_collection("Pict_1*")

In [ ]:
im = imc[0]

In [ ]:
plt.imshow(im)

In [ ]:
im_gray = (127. * im[:,:,0] + 71. * im[:, :, 1] + 35. * im[:, :, 2]) / 255
#im_gray = skimage.exposure.equalize_hist(im_gray)
plt.imshow(im_gray, cmap = 'gray')

In [ ]:
def threshold(i_):
    thr = skimage.filters.threshold_otsu(i_) * 1.1
    return(i_ > thr)
im_thr = threshold(im_gray)
plt.imshow(im_thr, cmap='gray')

In [ ]:
def find_edges(im_gray):
    k = np.array([[1,2,0,-2,-1],[4,8,0,-8,-4],[6,12,0,-12,-6],[4,8,0,-8,-4],[1,2,0,-2,-1]])
    r1 = ndimage.convolve(im_gray, k, mode='constant', cval=0.0)
    r2 = ndimage.convolve(im_gray, k.T, mode='constant', cval=0.0)
    r = np.maximum(r1, r2)
    return(r)

r = find_edges(im_thr)
r = skimage.exposure.equalize_hist(r)

In [ ]:
plt.imshow(np.abs(r - 0.5) ** 2, cmap='gray')

In [ ]:
from skimage.feature import blob_dog, blob_log, blob_doh

In [ ]:
im_thr

In [ ]:
blobs_log = blob_log(image_gray, max_sigma=30, num_sigma=10, threshold=.1)

# Compute radii in the 3rd column.
blobs_log[:, 2] = blobs_log[:, 2] * sqrt(2)

blobs_dog = blob_dog(image_gray, max_sigma=30, threshold=.1)
blobs_dog[:, 2] = blobs_dog[:, 2] * sqrt(2)

blobs_doh = blob_doh(image_gray, max_sigma=30, threshold=.01)

blobs_list = [blobs_log, blobs_dog, blobs_doh]
colors = ['yellow', 'lime', 'red']
titles = ['Laplacian of Gaussian', 'Difference of Gaussian',
          'Determinant of Hessian']
sequence = zip(blobs_list, colors, titles)

fig, axes = plt.subplots(1, 3, figsize=(9, 3), sharex=True, sharey=True,
                         subplot_kw={'adjustable': 'box-forced'})
ax = axes.ravel()

for idx, (blobs, color, title) in enumerate(sequence):
    ax[idx].set_title(title)
    ax[idx].imshow(image, interpolation='nearest')
    for blob in blobs:
        y, x, r = blob
        c = plt.Circle((x, y), r, color=color, linewidth=2, fill=False)
        ax[idx].add_patch(c)
    ax[idx].set_axis_off()

plt.tight_layout()
plt.show()